### Set cover problem 

Given a set of elements {1, 2, …, n} (called the universe) and a collection S of m subsets whose union equals the universe, the set cover problem is to identify the smallest sub-collection of S whose union equals the universe.Given a set of elements {1, 2, …, n} (called the universe) and a collection S of m subsets whose union equals the universe, the set cover problem is to identify the smallest sub-collection of S whose union equals the universe.

In [330]:
from random import random, choice, randint
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue, deque
from copy import  copy
from math import ceil 
import numpy as np
from tqdm.auto import tqdm

### Setting problem parameters

In [331]:
PROBLEM_SIZE  = 100 #elements to cover
NUM_SETS = 200
SETS =  tuple(np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)])  for _ in range(NUM_SETS) )
    #the value True means that the set contains the element
    #we randomly create NUM_SETS sets of PROBLEM_SIZE elements (True/False)\

#print('Problem size:', SETS)
State = namedtuple('State', ['taken', 'not_taken'])

In [332]:
#Function to check all the elements are covered

def goal_check(state):
    return np.all(covered(state))

def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

assert goal_check( ##check if taking all sets a solution exists
    State(set(range(NUM_SETS)), set())
), "Probelm not solvable"

## A* algorithm
The heuristic function should be always optimistic and respect some constraint to provide the best solution. 

In [333]:
#using the distance from the goal as a heuristic
def heuristic1(state):
    return PROBLEM_SIZE - len(state.taken)

#This heuristic does not provide always the best solution. Indeed it is not admissible becuase it can overestimate the cost to reach the goal
#I tried many different run using the breadth-first to find the minimun number of sets to reach the goal 
#and not always this heuristic provides the best solution.

In [334]:
#find an optimistic solution 
#i take dynamically the sets with the most elements covered
#how many max sets are necessary to cover all the current uncovered elements? 

def heuristic2(state : State):
    already_covered = covered(state) 
    if np.all(already_covered):
        return 0
    not_already_covered = np.logical_not(already_covered)   
    max_val = max(sum(np.logical_and(SETS[i], not_already_covered)) for i in state.not_taken)
    #max_val is the maximum number of uncovered element covered by a single element in the not_taken group
    return ceil(sum(not_already_covered)/max_val) if max_val!= 0 else PROBLEM_SIZE
    

In [335]:
#it is possible to define many cost function 
def actual_cost(state): 
    return len(state.taken)

def distance_from_the_goal(state):
    return PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )) 
        

def actual_cost2(state : State) :  
    return len(state.taken) + distance_from_the_goal(state)
###after some run, this cost function in not good to findo always the minimum number of elements

In [336]:
#define the heuristic to use
heuristic = heuristic2

actual_cost = actual_cost2


def a_star(state): 
    print("current_state taken : " , state.taken, "cost : ", actual_cost(state), " h :" ,heuristic(state))
    return actual_cost(state) + heuristic(state)

In [337]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))

frontier.put((a_star(state), state))

steps = 0
weight , current_state = frontier.get()

with tqdm(total=None) as pbar: 
    while not goal_check(current_state): 
        steps += 1
        for action in current_state.not_taken:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((a_star(new_state), new_state))     
        weight, current_state = frontier.get()
        print("Next step")
        print("current_state taken : " , current_state.taken, "weight : ", weight)
        pbar.update(1)

print(f'Solution found in {steps} steps and {len(current_state.taken)} tiles')
print(f'Final state: {current_state.taken}')
print(goal_check(current_state))


current_state taken :  set() cost :  100  h : 3


0it [00:00, ?it/s]

current_state taken :  {0} cost :  67  h : 3
current_state taken :  {1} cost :  78  h : 3
current_state taken :  {2} cost :  73  h : 3
current_state taken :  {3} cost :  77  h : 3
current_state taken :  {4} cost :  77  h : 3
current_state taken :  {5} cost :  72  h : 3
current_state taken :  {6} cost :  68  h : 3
current_state taken :  {7} cost :  72  h : 3
current_state taken :  {8} cost :  64  h : 3
current_state taken :  {9} cost :  72  h : 3
current_state taken :  {10} cost :  68  h : 2
current_state taken :  {11} cost :  71  h : 3
current_state taken :  {12} cost :  68  h : 2
current_state taken :  {13} cost :  67  h : 3
current_state taken :  {14} cost :  66  h : 3
current_state taken :  {15} cost :  70  h : 3
current_state taken :  {16} cost :  60  h : 3
current_state taken :  {17} cost :  70  h : 2
current_state taken :  {18} cost :  64  h : 2
current_state taken :  {19} cost :  75  h : 3
current_state taken :  {20} cost :  66  h : 3
current_state taken :  {21} cost :  71  h : 

1it [00:00,  1.57it/s]

current_state taken :  {190} cost :  67  h : 3
current_state taken :  {191} cost :  65  h : 2
current_state taken :  {192} cost :  71  h : 3
current_state taken :  {193} cost :  79  h : 3
current_state taken :  {194} cost :  61  h : 2
current_state taken :  {195} cost :  70  h : 3
current_state taken :  {196} cost :  76  h : 3
current_state taken :  {197} cost :  68  h : 3
current_state taken :  {198} cost :  67  h : 3
current_state taken :  {199} cost :  76  h : 3
Next step
current_state taken :  {38} weight :  60
current_state taken :  {0, 38} cost :  38  h : 2
current_state taken :  {1, 38} cost :  45  h : 3
current_state taken :  {2, 38} cost :  41  h : 3
current_state taken :  {3, 38} cost :  45  h : 3
current_state taken :  {4, 38} cost :  43  h : 3
current_state taken :  {5, 38} cost :  43  h : 3
current_state taken :  {38, 6} cost :  36  h : 2
current_state taken :  {38, 7} cost :  42  h : 2
current_state taken :  {8, 38} cost :  35  h : 2
current_state taken :  {9, 38} cost : 

...

current_state taken :  {38, 6, 167, 41, 54} cost :  6  h : 1
current_state taken :  {6, 38, 168, 41, 54} cost :  8  h : 1
current_state taken :  {38, 6, 169, 41, 54} cost :  8  h : 1
current_state taken :  {38, 6, 41, 170, 54} cost :  9  h : 1
current_state taken :  {38, 6, 41, 171, 54} cost :  8  h : 1
current_state taken :  {38, 6, 41, 172, 54} cost :  9  h : 1
current_state taken :  {38, 6, 41, 173, 54} cost :  9  h : 1
current_state taken :  {6, 38, 41, 174, 54} cost :  8  h : 1
current_state taken :  {38, 6, 41, 175, 54} cost :  8  h : 1
current_state taken :  {6, 38, 41, 176, 54} cost :  9  h : 1
current_state taken :  {38, 6, 41, 177, 54} cost :  9  h : 1
current_state taken :  {38, 6, 41, 178, 54} cost :  7  h : 1
current_state taken :  {38, 6, 41, 179, 54} cost :  8  h : 1
current_state taken :  {38, 6, 41, 180, 54} cost :  9  h : 2
current_state taken :  {38, 6, 41, 181, 54} cost :  10  h : 2
current_state taken :  {6, 38, 41, 54, 182} cost :  8  h : 1
current_state taken :  

6it [00:03,  1.90it/s]

current_state taken :  {38, 6, 41, 106, 85, 54} cost :  7  h : 1
current_state taken :  {38, 6, 41, 106, 86, 54} cost :  7  h : 1
current_state taken :  {6, 38, 41, 106, 54, 87} cost :  7  h : 1
current_state taken :  {38, 6, 41, 106, 54, 88} cost :  7  h : 1
current_state taken :  {38, 6, 41, 106, 54, 89} cost :  7  h : 1
current_state taken :  {38, 6, 41, 106, 54, 90} cost :  6  h : 0
current_state taken :  {38, 6, 41, 106, 54, 91} cost :  7  h : 1
current_state taken :  {38, 6, 41, 106, 54, 92} cost :  6  h : 0
current_state taken :  {38, 6, 41, 106, 54, 93} cost :  6  h : 0
current_state taken :  {38, 6, 41, 106, 54, 94} cost :  7  h : 1
current_state taken :  {6, 38, 41, 106, 54, 95} cost :  7  h : 1
current_state taken :  {96, 38, 6, 41, 106, 54} cost :  7  h : 1
current_state taken :  {97, 38, 6, 41, 106, 54} cost :  6  h : 0
current_state taken :  {98, 38, 6, 41, 106, 54} cost :  7  h : 1
current_state taken :  {99, 38, 6, 41, 106, 54} cost :  7  h : 1
current_state taken :  {1

6it [00:03,  1.76it/s]

Solution found in 6 steps and 6 tiles
Final state: {0, 38, 6, 41, 106, 54}


### BREADTH-FIRST SEARCH
to verify the optimal number of sets

In [338]:
return
frontier = deque()
state = State(set(), set(range(NUM_SETS)))
frontier.append(state)

counter = 0
current_state = frontier.popleft()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.append(new_state)
        current_state = frontier.popleft()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")
print(f'Final state: {current_state.taken}')

SyntaxError: 'return' outside function (582259064.py, line 1)